### A360AI
Ensure that the A360AI object is loaded

In [ ]:
a360ai

This is the project associated with the workspace.

In [ ]:
a360ai.project_name

This is the workspace flavor. It can be any one of 'spark', 'scipy', 'pytorch', 'tensorflow', 'huggingface', or 'xgboost'.

In [ ]:
import os
image_flavor = os.environ["A360_IMAGE"].split("-")
if image_flavor[-2] == "huggingface":
    flavor = "huggingface"
else:
    flavor = image_flavor[-1]
flavor

### Data Repos
Data Repos are A360AI objects used to read and write datasets. A project can be associated with multiple datarepos but a datarepo is always associated with a single project.

List the datarepos associated with the current project.

In [ ]:
datarepos = a360ai.list_datarepos()

if datarepos.shape[0] == 0:
    raise Exception(f"No data repos associated with the project {a360ai.project_name}. Please add a datarepo.")
datarepos

#### Select a datarepo
Please set the *datarepo_name* variable to a valid string. The ``set_default_datarepo`` method needs to be used in order to access the datasets stored in the specific datarepo.

In [ ]:
### Can use "Health Byte - Dev"

datarepo_name = <DATAREPO_NAME> 
a360ai.set_default_datarepo(datarepo_name)

List the datasets stored in the current datarepo.

In [ ]:
datasets = a360ai.list_datasets()

if datasets.shape[0] == 0:
    raise Exception(f"No datasets associated with the datarepo {datarepo_name}. Please add a dataset.")
datasets

#### Select the model type

Please specify the type of task to be performed. Options are limited to "classification" and "regression".

In [ ]:
model_type = <MODEL_TYPE> 

#### Select the dataset
Please set the *dataset_name* variable to a valid string. 

In [ ]:
### The datasets "merged_anonymous.parquet" (classification)
### or "machine_dataset.csv" (regression) can be used.
### Both datasets are in "Health Byte - Dev" datarepo.

dataset_name = <DATASET_NAME> 

df = a360ai.load_dataset(dataset_name)

#### Specify the target column 
Please set the target column out of the following columns

In [ ]:
df.columns.values.tolist()

In [ ]:
### "lifestyle" if using "merged_anonymous.parquet" 
### or "published_relative_performance" if using "machine_dataset.csv"

target_column = <TARGET_COLUMN> 

#### Please provide the number of classes
If the model_type is classification, please specify the number of classes present. An integer should be assigned to the variable ``num_classes``.

In [ ]:
### 3 if using "merged_anonymous.parquet"

num_classes = <NUM_CLASSES>

#### Split the dataset into train/test data

The ``get_dummies`` method yields one-hot encoded vectors for classification tasks.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X, y = df.drop(target_column,axis=1), df[target_column]

if model_type == "classification":
    y = pd.get_dummies(y, prefix=target_column)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, test_size=0.1)
    
num_features = X_train.shape[1]

### A360AI Model

Create/retrieve an A360AI model. The following arguments need to be passed to the ``get_or_create_model`` method: model_name, version and model_type (default is "classification").

In [ ]:
from uuid import uuid4
unique = str(uuid4())[-6:]

model = a360ai.get_or_create_model(
    model_name=f"Sample Notebook {flavor} {model_type} - {unique}", 
    version="1.0", 
    model_type=model_type
)

### A360AI Experiment

Create/retrieve an A360AI experiment. The following arguments need to be passed to the ``get_or_create_experiment`` method of the Model object: experiment_name, model_flavor, train_features and train_target. 

In [ ]:
experiment = model.get_or_create_experiment(
    experiment_name=f"sample_notebook_experiment{unique}",
    model_flavor=flavor,
    train_features=X_train,
    train_target=y_train,
)

#### View the experiments listed with the model

In [ ]:
model.list_experiments()

#### Choose the candidate model to be trained.

This model will be trained on the train dataset and evaluated on the test dataset. The candidate model will also be logged later after it has been trained.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
    
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.x=torch.from_numpy(X_train).float()
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index].float(), self.y[index].float()
    def __len__(self):
        return self.len
    
data_set=Data(X_train, y_train)
trainloader=DataLoader(dataset=data_set, batch_size=32)
    
class create_torch_model(nn.Module):
    def __init__(self, num_feature, num_class, model_type):
        super(create_torch_model, self).__init__()

        self.layer_1 = nn.Linear(num_feature, 4)
            
        if model_type == "classification":
            self.layer_2 = nn.Linear(4, num_class)
        else:
            self.layer_2 = nn.Linear(4, 1)

        self.relu = nn.ReLU()
            
    def forward(self, inputs):
        x = self.layer_1(inputs)
        x = self.relu(x)
        x = self.layer_2(x)
            
        return x
        
candidate_model = create_torch_model(num_features, num_classes, model_type)

if model_type == "classification":
    criterion = nn.L1Loss()
elif model_type == "regression":
    criterion = nn.MSELoss()
optimizer = optim.Adam(candidate_model.parameters())

### Create an A360AI Run

Use the Run to log the trained model, metadata, metrics and hyperparameters.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error


with experiment.run_experiment() as run:
    n_epochs=10
    loss_list=[]

    candidate_model.train()
    
    for epoch in range(n_epochs):
        for x, y in trainloader:
            optimizer.zero_grad()
            z=candidate_model(x)
            loss=criterion(z,y)
            loss.backward() 
            optimizer.step()

            loss_list.append(loss.data)
    
    
    train_data = torch.from_numpy(X_train).float()
    test_data = torch.from_numpy(X_test).float()

    y_pred_train = candidate_model(train_data)
    y_pred_test = candidate_model(test_data)
        
    if model_type == "classification":
        _, y_pred_train = torch.max(y_pred_train, 1)         
        _, y_pred_test = torch.max(y_pred_test, 1)
            
        y_train = np.argmax(y_train, 1)
        y_test = np.argmax(y_test, 1)

        train_score = accuracy_score(y_train, y_pred_train)            
        test_score = accuracy_score(y_test, y_pred_test)
    else:
        y_pred_train = y_pred_train.detach().numpy()
        y_pred_test = y_pred_test.detach().numpy()
            
        train_score = mean_squared_error(y_train, y_pred_train)
        test_score = mean_squared_error(y_test, y_pred_test)
            
    metrics = {
        "train_score": train_score,  
        "test_score": test_score,  
    }
        
    run.log_metadata({
        "notes": "This model was generated with extreme care."
    })
    run.log_metrics(metrics)
    run.log_hyperparameters({
        "foo": 1
    })
    run.log_model(candidate_model)

#### View the runs associated with the experiment. 
If the code above has been executed successfully, there will be a new entry in the dataframe returned by the ``list_runs`` method.

In [ ]:
experiment.list_runs()